In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json

In [2]:
DATAPATH = 'data'
labels = pd.read_csv(os.path.join(DATAPATH,'labels.csv')) 

label_filenames = labels['filename'].values
new_label_filenames = [l.replace(":","_") for l in label_filenames]
labels['filename'] = new_label_filenames


#We see that fuel type, coordinates, weather data and generation output rate
classification_labels = labels[['filename','fuel_type']].copy()
regression_labels = labels[['filename','gen_output']].copy()
weather_data = labels[['filename','temp','wind-u','wind-v','humidity']].copy()

labels

,filename,country,fuel_type,lat,lon,temp,wind-u,wind-v,humidity,gen_output
0,0000__S2B-MSIL2A-ST20200122T111720-N0213-R137-...,NaN,Fossil Gas,49.480000,0.220000,278.305176,0.004515,4.226044,82.740540,522.00
1,0046__S2B-MSIL2A-ST20200113T104630-N0213-R008-...,NaN,Fossil Coal-derived gas,51.492197,6.727517,281.209442,0.006401,6.036209,95.168945,261.75
2,0002__S2B-MSIL2A-ST20200122T111720-N0213-R137-...,NaN,Fossil Hard coal,49.473119,0.145785,278.305176,0.004515,4.226044,82.740540,490.00
3,0000__S2A-MSIL2A-ST20200104T110726-N0213-R094-...,NaN,Fossil Gas,49.480000,0.220000,275.573242,0.004171,-3.539032,91.366272,142.00
4,0046__S2B-MSIL2A-ST20200113T104631-N0213-R008-...,NaN,Fossil Coal-derived gas,51.492197,6.727517,281.209442,0.006401,6.036209,95.168945,261.75
...,...,...,...,...,...,...,...,...,...,...
2126,46405_2019-10-02T10_35_39.153Z_0.tif,Germany,Fossil Hard coal,54.142813,12.132942,284.489777,6.844803,-2.311661,60.229195,339.23
2127,46405_2019-10-07T10_35_40.952Z_0.tif,Germany,Fossil Hard coal,54.142813,12.132942,279.732239,0.855591,-1.933838,74.679741,504.63
2128,2450_2019-10-12T10_37_13.780Z_0.tif,Germany,Fossil Hard coal,48.719120,9.371298,293.213806,1.626221,2.559738,57.040428,0.00
2129,2450_2019-10-14T10_27_19.241Z_0.tif,Germany,Fossil Hard coal,48.719120,9.371298,295.094391,-1.124039,-0.288925,58.943527,0.00


In [3]:
def walk_seg_labels():
    train120 = []
    train300 = []
    val = []
    filenames = []

    for roots,dirs,files in os.walk(os.path.join(DATAPATH,'segmentation_labels')):
        for f in files:
            try:
                with open(os.path.join(roots,f),'r') as file:
                    jsonfile = json.load(file)
                    # print(roots)
                    # print(os.path.join(roots,files[0]))
                    file.close()
                
                filenames.append(f.split('_features.json')[0])
                # filenames.append(f.split('.')[0])
                if '120x120' in roots:
                    train120.append(jsonfile)
                elif '300x300' in roots:
                    train300.append(jsonfile)
                elif 'validation' in roots:
                    val.append(jsonfile)
                else:
                    print(jsonfile)
                    raise ValueError("Somehow an invalid JSON file was read.")

            except (IndexError,FileNotFoundError,json.JSONDecodeError):
                print(f'ERROR FOR: {f}')
                continue

    return train120,train300,val,filenames

def walk_images():
    filenames = []
    for roots,dirs,files in os.walk(os.path.join(DATAPATH,'images')):
        for f in files:
            if 'DS' in f or f in filenames:
                continue
            filenames.append(f)
    return filenames

image_filenames = walk_images()
len(image_filenames)

2125

In [4]:
train120,train300,val,seg_filenames = walk_seg_labels()
len(val),len(train120),len(train300),len(val)+len(train120)+len(train300),len(seg_filenames)

ERROR FOR: .DS_Store
ERROR FOR: ._.DS_Store


(73, 560, 938, 1571, 1571)

In [5]:
filename_mask = np.isin(new_label_filenames,image_filenames)
print(sum(filename_mask))
labels[filename_mask]

2125


,filename,country,fuel_type,lat,lon,temp,wind-u,wind-v,humidity,gen_output
0,0000__S2B-MSIL2A-ST20200122T111720-N0213-R137-...,NaN,Fossil Gas,49.480000,0.220000,278.305176,0.004515,4.226044,82.740540,522.00
1,0046__S2B-MSIL2A-ST20200113T104630-N0213-R008-...,NaN,Fossil Coal-derived gas,51.492197,6.727517,281.209442,0.006401,6.036209,95.168945,261.75
2,0002__S2B-MSIL2A-ST20200122T111720-N0213-R137-...,NaN,Fossil Hard coal,49.473119,0.145785,278.305176,0.004515,4.226044,82.740540,490.00
3,0000__S2A-MSIL2A-ST20200104T110726-N0213-R094-...,NaN,Fossil Gas,49.480000,0.220000,275.573242,0.004171,-3.539032,91.366272,142.00
4,0046__S2B-MSIL2A-ST20200113T104631-N0213-R008-...,NaN,Fossil Coal-derived gas,51.492197,6.727517,281.209442,0.006401,6.036209,95.168945,261.75
...,...,...,...,...,...,...,...,...,...,...
2126,46405_2019-10-02T10_35_39.153Z_0.tif,Germany,Fossil Hard coal,54.142813,12.132942,284.489777,6.844803,-2.311661,60.229195,339.23
2127,46405_2019-10-07T10_35_40.952Z_0.tif,Germany,Fossil Hard coal,54.142813,12.132942,279.732239,0.855591,-1.933838,74.679741,504.63
2128,2450_2019-10-12T10_37_13.780Z_0.tif,Germany,Fossil Hard coal,48.719120,9.371298,293.213806,1.626221,2.559738,57.040428,0.00
2129,2450_2019-10-14T10_27_19.241Z_0.tif,Germany,Fossil Hard coal,48.719120,9.371298,295.094391,-1.124039,-0.288925,58.943527,0.00


In [13]:
def cross_ref_seg_with_labels(seg_filenames=seg_filenames,label_filenames=new_label_filenames):
    new_label_filenames = [l.split('.')[0] for l in label_filenames]
    print(len(new_label_filenames))
    print(len(seg_filenames))
    print(len(new_label_filenames) - len(seg_filenames))
    return np.isin(seg_filenames,new_label_filenames).sum()

cross_ref_seg_with_labels()

2131
1571
560


940

In [12]:
sum([new_label_filenames.count(l) > 1 for l in image_filenames])

0